#### 1. ClassLoader 的 parent 层级   
jvm 中的类都是由类加载器加载的， 类加载器本身也是一个对象，所以我们说的 ClassLoader 是一个可以加载类的`对象`。jvm 内置了三个进程内唯一的类加载器对象，此外用户也可以用 `new` 实例化自己的类加载器对象。jvm 自身维护了`内置的三个类加载器对象`和`自己实例化的类加载器对象`之间的 parent 层级关系，表现为:
* 由 C++ 实现的顶级 `BootStrap加载器`：   
  因为是 C++ 实现，如果打印这个类加载器对象，表示为 null。负责加载 `$JAVA_HOME/lib` 目录下的 jar 包中的 class  
* Java 实现的 `Extension 加载器`：    
  用 sun.misc.Launcher 的内部类 `ExtClassLoader` 表示,负责加载`$JAVA_HOME/lib/ext`目录下的jar包     
* Java 实现的 Application 加载器：   
  用 sun.misc.Launcher 的内部类 `AppClassLoader` 表示， 负责加载`$classpath`里的类。因为 `ClassLoader#getSystemClassLoader()` 方法返回的就是 AppClassLoader 对象，所以又叫系统类加载器
  
ClassLoader 对象的层级关系不是通过继承实现的，而是通过组合实现，用 `parent 属性`表示，如下代码验证了这种层级关系
```java
public static void testLoaderLevel(){
// MyClassLoader 是一个自定义类加载器，自定义方法后面介绍
    ClassLoader myLoader1 = new MyClassLoader();
    System.out.println(myLoader1); // MyClassLoader@548c4f57 

    ClassLoader parent1 = myLoader1.getParent();
    System.out.println(parent1);   // sun.misc.Launcher$AppClassLoader@18b4aac2

    ClassLoader parent2 = parent1.getParent();
    System.out.println(parent2);   // sun.misc.Launcher$ExtClassLoader@1218025c

    ClassLoader parent3 = parent2.getParent();
    System.out.println(parent3);   // null
    
    System.out.println(ClassLoader.getSystemClassLoader() == parent1); // true (反映了三个内置类加载器的全局唯一)
}
```

#### 2. 类加载器的 parent 委派机制     
java 设计了类加载器对象的基类 `ClassLoader`，是上述 ext, app, 自定义类加载器的基类，该类中定义了类加载的双亲委派模式。  
首先，jvm 用一个类加载对象加载类时，以 `ClassLoader#loadClassInternal()` 方法为入口， 该方法只调用了 `ClassLoader#loadClass()` 方法，该方法是线程安全的。  
```java
// ClasLoader.java

// This method is invoked by the virtual machine to load a class.
private Class<?> loadClassInternal(String name)
    throws ClassNotFoundException
{
    // For backward compatibility, explicitly lock on 'this' when
    // the current class loader is not parallel capable.
    if (parallelLockMap == null) {
        synchronized (this) {
             return loadClass(name);
        }
    } else {
        return loadClass(name);
    }
}
```
接着，在 `ClassLoader#loadClass()` 方法中，实现了类加载器的双亲委派机制： 递归调用 parent 的 loadClass() 方法：   
* 先看用类加载对象，看该类名是否已经被加载  
* 如果未被加载，则递归调用 parent 类加载器加载，否则调用自身的 `findClass()` 进行加载。

```java
// ClassLoader.java

protected Class<?> loadClass(String name, boolean resolve)
        throws ClassNotFoundException
{
    synchronized (getClassLoadingLock(name)) {
        // First, check if the class has already been loaded
        Class<?> c = findLoadedClass(name);
        if (c == null) {
            long t0 = System.nanoTime();
            try {
                if (parent != null) {
                    c = parent.loadClass(name, false);
                } else {
                    c = findBootstrapClassOrNull(name);
                }
            } catch (ClassNotFoundException e) {
                // ClassNotFoundException thrown if class not found
                // from the non-null parent class loader
            }

            if (c == null) {
                // If still not found, then invoke findClass in order
                // to find the class.
                long t1 = System.nanoTime();
                c = findClass(name);

                // this is the defining class loader; record the stats
                sun.misc.PerfCounter.getParentDelegationTime().addTime(t1 - t0);
                sun.misc.PerfCounter.getFindClassTime().addElapsedTimeFrom(t1);
                sun.misc.PerfCounter.getFindClasses().increment();
            }
        }
        if (resolve) {
            resolveClass(c);
        }
        return c;
    }
}

protected Class<?> findClass(String name) throws ClassNotFoundException {
    throw new ClassNotFoundException(name);
}
```

这里就暴露了自定义类加载器的实现方法：  
* 首先，让自定义的类加载器类 `extends ClassLoader`
* 如果想保留双亲委派机制，只`覆盖 findClass()` 方法即可。   
  `protected final Class<?> defineClass(String name, byte[] b, int off, int len)` 方法将字节数组转换为 `Class<?>` 对象的方法，所以只需在 findClass 中获取字节数组，再调用 ClassLoader#defineClass() 方法即可。  
* 如果想打破双亲委派机制，就去覆盖`loadClass()`方法。

如下，自定义类加载器的方法

```java
class MyClassLoader extends ClassLoader{
    @Override
    public Class<?> loadClass(String name) throws ClassNotFoundException {
        if ("ClassA".equals(name)){
            try{
                InputStream is = getClass().getResourceAsStream("/ClassA.class");

                byte[] bytes = new byte[is.available()];
                is.read(bytes);
                return defineClass(name,bytes,0,bytes.length);
            }catch (IOException ignored){}
        }else {
            return super.loadClass(name);
        }
        return null;
    };
}
```

#### 3. 子类加载器可以访问 parent 类加载加载的类，parent 类加载器加载的类不能访问子类加载器加载的类  
[https://blog.csdn.net/qq_20846769/article/details/100026116?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522162480761116780271595492%2522%252C%2522scm%2522%253A%252220140713.130102334.pc%255Fall.%2522%257D&request_id=162480761116780271595492&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~first_rank_v2~rank_v29-1-100026116.first_rank_v2_pc_rank_v29&utm_term=classloader%E4%B8%8E%E5%91%BD%E5%90%8D%E7%A9%BA%E9%97%B4&spm=1018.2226.3001.4187](https://blog.csdn.net/qq_20846769/article/details/100026116?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522162480761116780271595492%2522%252C%2522scm%2522%253A%252220140713.130102334.pc%255Fall.%2522%257D&request_id=162480761116780271595492&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~first_rank_v2~rank_v29-1-100026116.first_rank_v2_pc_rank_v29&utm_term=classloader%E4%B8%8E%E5%91%BD%E5%90%8D%E7%A9%BA%E9%97%B4&spm=1018.2226.3001.4187)

#### 4. Thread 类中的 contextClassLoader
```java
private ClassLoader contextClassLoader
```

#### 6. jdbc4.0 以后使用 SPI 跳过 Class.forName

#### 5. ServiceLoader 与 SPI



#### 7. springboot 自动装配与 SPI



### 一. Java类加载步骤
#### (1) Java加载.class文件后到类生成,消亡经历了什么过程 ?
1. 加载
2. 验证 : 验证.class文件中的内容是否符合jvm虚拟机规范  
   **准备** : 开辟内存空间来存储这个类和其类变量(就是内部的static变量), 为对象的成员变量赋"0"值  
   解析
3. **初始化** : 
    1. 如果该类有父类, 先加载, 验证, 解析父类
    2. 再对成员变量赋代码这种声明的真正的值  
4. 使用
5. 卸载

#### (2) 什么时候会加载.class文件
当使用到某个类时, 类加载器会对.class文件进行加载,验证,准备,解析,初始化,使用


### 二. 类加载器和双亲委派机制

#### 2.1 Java中的类加载器分类(依据.class文件的位置不同划分)
1. BootStrap加载器  
   负责加载`$JAVA_HOME/lib`目录下的jar包, 或通过`-Xbootclasspath`指定路径下的jar包

2. Extension加载器   
   负责加载`$JAVA_HOME/lib/ext`目录下的jar包

3. Application加载器   
   加载`$classpath`里的类, 直到applicaiton加载器, 才是加载自己写的代码
   
4. 自定义类加载器  
   
#### 2.2 类加载器的双亲委派机制
1. Java类加载器时层级结构   
   如上所述, java的类加载器时分层的, 形成了自顶向下的亲子结构. 
   <img src="img/classloader.png" width="20%">
   
2. 类加载器的双亲委派机制  
    * 每层的类加载器都会优先让父一级的类加载器加载.如果父一级的加载器找不到.class文件, 再由自己加载.  
    * 双亲委派机制, 并不是强制性要求, 而是jvm设计者推荐的一种类加载方式. 这种做法的好处是, 无论是哪个类加载器加载的.class文件, 最终都会被指派给同一个祖先类加载器加载    
     
3. JVM在搜索类的时候，又是如何判定两个class是相同的呢？  
两个类的`类名`和`类加载器`都一样时, jvm才判定这事同一个类

4. 线程的 `context class loader` 和 普通的`类 classloader` 有什么不同
```java
Thread.currentThread().getContextClassLoader(); - 线程的上下文类加载器
this.getClass().getClassLoader();               - 类的类加载器
```
有一个规则 : 每个类都是用自己的 classLoader 去加载其他类. 假若 ClassA.class 引用了 ClassB.class, 那么 ClassB 需要在 ClassA 的 classLoader 的 classpath 下; `线程上下文类加载器` 是当前线程的 classLoader. 如果一个对象由 classLoaderC 创建传给了 classLoaderD 拥有的线程下使用, 则该对象要想使用自己的 classLoader 加载不到的资源时, 就要使用 `Thread.currentThread().getContextClassLoader()` 加载
   
     
5. spark 中加载 "hive-site.xml" 的方式
```scala
Option(Thread.currentThread().getContextClassLoader).getOrElse(getSparkClassLoader)
def getSparkClassLoader: ClassLoader = getClass.getClassLoader
```